# SQL


In [3]:
import qrcode
import os

def generate_qr_codes(base_url, route_id, stop_count, out_dir="qrcodes"):
    os.makedirs(out_dir, exist_ok=True)

    for stop_order in range(1, stop_count + 1):
        url = f"{base_url}/routes/{route_id}/stop/{stop_order}"
        img = qrcode.make(url)
        filename = os.path.join(out_dir, f"route{route_id}_stop{stop_order}.png")
        img.save(filename)
        print("已產生:", filename, "→", url)

# 使用範例
if __name__ == "__main__":
    # 你的 ngrok host，或正式網域
    base = "https://f1c2fbf338af.ngrok-free.app"
    generate_qr_codes(base_url=base, route_id=2, stop_count=8)


已產生: qrcodes\route2_stop1.png → https://f1c2fbf338af.ngrok-free.app/routes/2/stop/1
已產生: qrcodes\route2_stop2.png → https://f1c2fbf338af.ngrok-free.app/routes/2/stop/2
已產生: qrcodes\route2_stop3.png → https://f1c2fbf338af.ngrok-free.app/routes/2/stop/3
已產生: qrcodes\route2_stop4.png → https://f1c2fbf338af.ngrok-free.app/routes/2/stop/4
已產生: qrcodes\route2_stop5.png → https://f1c2fbf338af.ngrok-free.app/routes/2/stop/5
已產生: qrcodes\route2_stop6.png → https://f1c2fbf338af.ngrok-free.app/routes/2/stop/6
已產生: qrcodes\route2_stop7.png → https://f1c2fbf338af.ngrok-free.app/routes/2/stop/7
已產生: qrcodes\route2_stop8.png → https://f1c2fbf338af.ngrok-free.app/routes/2/stop/8


In [4]:
import requests
import json
import hashlib
from base64 import b64encode, b64decode
from Crypto.Cipher import AES
from urllib.parse import quote

# ==== 你的參數 (請替換成表格上的資料) ====
MERCHANT_ID = "106952201270001"   # RG商代
TERMINAL_ID = "70002146"          # RG端代
STORE_CODE  = "e4f76eb8-d236-45fb-984b-92b2d1912492"  # 店家識別碼
KEY = bytes.fromhex("3abae022acd6fc873821411c0b402c0fbe90d90bdda295ed15296f8ae465bf8b")  # 端末設備Key (hex)
IV  = bytes.fromhex("12fe9f5e0c3cc7c664894f19ba265050")  # 端末設備IV (hex，記得補齊完整)
LAYMON = "https://iqrc.epay365.com.tw/"
# ==== 測試交易資料 ====
payload = {
    "set_price": "100",
    "pos_order_number": "TEST12345",   # 自訂訂單編號，不可重複
    "callback_url": "https://yourdomain.com/callback",
    "return_url": "https://yourdomain.com/return"
}

# --- AES256-CBC 加密 ---
def pad(s):
    pad_len = 16 - (len(s) % 16)
    return s + chr(pad_len) * pad_len

def unpad(s):
    return s[:-ord(s[-1])]

def encrypt_aes(data: str, key: bytes, iv: bytes) -> str:
    cipher = AES.new(key, AES.MODE_CBC, iv)
    ct_bytes = cipher.encrypt(pad(data).encode("utf-8"))
    return b64encode(ct_bytes).decode("utf-8")

def decrypt_aes(enc: str, key: bytes, iv: bytes) -> str:
    cipher = AES.new(key, AES.MODE_CBC, iv)
    pt = cipher.decrypt(b64decode(enc))
    return unpad(pt.decode("utf-8"))

# --- 加密交易資料 ---
json_str = json.dumps(payload, separators=(',', ':'))
TransactionData = encrypt_aes(json_str, KEY, IV)
HashDigest = hashlib.sha256(TransactionData.encode("utf-8")).hexdigest()

print("加密後 TransactionData:", TransactionData)
print("校驗碼 HashDigest:", HashDigest)

# --- 發送 API (線上付款) ---
url = f"https://[雷門API網址]/calc/pay_encrypt/{STORE_CODE}"
params = {
    "TransactionData": quote(TransactionData),  # URL encode
    "HashDigest": HashDigest
}

print("請求網址：", url, params)

# ⚠️ 測試時可先列印，不要真的發送
# response = requests.get(url, params=params)
# print(response.text)


加密後 TransactionData: LwIo/f0GsJQ1mLLlWKTqJsoi5qsxSApZKfjpXToaReHNFDMcw560TLVvIOVcIt6NDQyc8YwnCu8tECEJaTJqG1FP1rI/0wYlJeQR1p1Dw8CqA84ySf6IrrHROz3beuJ3QiZF7sYbCxglkz/jbpr9RxfUrt9RQQ8zf3ZmAcWLeMs9J6ryCBIfx+p3XWEk2T0jXyFz/um3DAOK9miKcbQpeA==
校驗碼 HashDigest: 98c15532d725f6135887ce75b867333697f89406bd253fe25192768854434556
請求網址： https://[雷門API網址]/calc/pay_encrypt/e4f76eb8-d236-45fb-984b-92b2d1912492 {'TransactionData': 'LwIo/f0GsJQ1mLLlWKTqJsoi5qsxSApZKfjpXToaReHNFDMcw560TLVvIOVcIt6NDQyc8YwnCu8tECEJaTJqG1FP1rI/0wYlJeQR1p1Dw8CqA84ySf6IrrHROz3beuJ3QiZF7sYbCxglkz/jbpr9RxfUrt9RQQ8zf3ZmAcWLeMs9J6ryCBIfx%2Bp3XWEk2T0jXyFz/um3DAOK9miKcbQpeA%3D%3D', 'HashDigest': '98c15532d725f6135887ce75b867333697f89406bd253fe25192768854434556'}


wacs.exe --target manual --host hualenbus.labelnine.app --validation cloudflare --validationmode dns-01 --store pemfiles --pemfilespath D:\certs --cloudflareapitoken Qhqik5_3YR4QhSn1xznX05_ftbzX9UWB0Egiernw


uvicorn Server:app --host 0.0.0.0 --port 8500 --ssl-keyfile "D:\SSL\hualenbus.labelnine.app-key.pem" --ssl-certfile "D:\SSL\hualenbus.labelnine.app-chain.pem"
